In [131]:
# 初始文档库
vocab={'h o l y </w>': 5, 'h o l i e r </w>': 2, 'n e w s t </w>': 6, 'w i d e s t </w>':3}

In [132]:
# 将所有的单词拆分成单个字符，并且在最后添加停止符</w>，同时标记词频
import collections
def get_tokens(vocab):
    tokens = collections.defaultdict(int)
    for word, freq in vocab.items():
        word_tokens = word.split()
        for token in word_tokens:
            tokens[token] += freq
    return tokens
#tokens = get_tokens(vocab)
#print('Tokens: {}'.format(tokens))
#print('Number of tokens: {}'.format(len(tokens)))

In [133]:
# 第3.1步：统计词典中连续字节对的出现频率，代码实现和执行一次的结果如下所示，
def get_stats(vocab):
    pairs = collections.defaultdict(int)
    print("vocab:",vocab)
    for word, freq in vocab.items():
        symbols = word.split()
        #print("symbols:",symbols)
        for i in range(len(symbols)-1):
            pairs[symbols[i],symbols[i+1]] += freq
    return pairs


In [134]:
# pairs = get_stats(vocab)
# print('pairs: {}'.format(pairs))

In [135]:
# 找到最高频率的连续字节对
# best = max(pairs, key=pairs.get)
# print('Best pair: {} count:{}'.format(best,pairs[best]))

In [136]:
import re
# 第3.3步：合成新的subword，代码实现和合成后的结果如下所示，通过正则匹配，将vocab中指定连续字节对进行合并。
def merge_vocab(pair, v_in):
    v_out = {}
    bigram = re.escape(' '.join(pair))
    p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
    for word in v_in:
        w_out = p.sub(''.join(pair), word)
        v_out[w_out] = v_in[word]
    return v_out

In [137]:
#vocab = merge_vocab(best, vocab)

In [138]:
print(vocab)

{'h o l y </w>': 5, 'h o l i e r </w>': 2, 'n e w s t </w>': 6, 'w i d e s t </w>': 3}


In [139]:
# 综合以上
for iter in range(10):
    print("iter:", iter)
    # 统计词典中连续字节对的出现频率
    pairs = get_stats(vocab)
    best = max(pairs, key=pairs.get)
    print('Best pair: {} count:{}'.format(best,pairs[best]))
    # 成新的subword，代码实现和合成后的结果如下所示，通过正则匹配，将vocab中指定连续字节对进行合并
    vocab = merge_vocab(best, vocab)
    print('new vocab:',vocab)
    # # 将所有的单词拆分成单个字符，同时标记词频(注意：这里每次计算token其实比较低效)
    tokens = get_tokens(vocab)
    print('Tokens: {}'.format(tokens))
    print('Number of tokens: {}'.format(len(tokens)),"\n\n")

iter: 0
vocab: {'h o l y </w>': 5, 'h o l i e r </w>': 2, 'n e w s t </w>': 6, 'w i d e s t </w>': 3}
Best pair: ('s', 't') count:9
new vocab: {'h o l y </w>': 5, 'h o l i e r </w>': 2, 'n e w st </w>': 6, 'w i d e st </w>': 3}
Tokens: defaultdict(<class 'int'>, {'h': 7, 'o': 7, 'l': 7, 'y': 5, '</w>': 16, 'i': 5, 'e': 11, 'r': 2, 'n': 6, 'w': 9, 'st': 9, 'd': 3})
Number of tokens: 12 


iter: 1
vocab: {'h o l y </w>': 5, 'h o l i e r </w>': 2, 'n e w st </w>': 6, 'w i d e st </w>': 3}
Best pair: ('st', '</w>') count:9
new vocab: {'h o l y </w>': 5, 'h o l i e r </w>': 2, 'n e w st</w>': 6, 'w i d e st</w>': 3}
Tokens: defaultdict(<class 'int'>, {'h': 7, 'o': 7, 'l': 7, 'y': 5, '</w>': 7, 'i': 5, 'e': 11, 'r': 2, 'n': 6, 'w': 9, 'st</w>': 9, 'd': 3})
Number of tokens: 12 


iter: 2
vocab: {'h o l y </w>': 5, 'h o l i e r </w>': 2, 'n e w st</w>': 6, 'w i d e st</w>': 3}
Best pair: ('h', 'o') count:7
new vocab: {'ho l y </w>': 5, 'ho l i e r </w>': 2, 'n e w st</w>': 6, 'w i d e st</w>'

In [141]:
# 编码的原理同样比较拗口，这里直接介绍代码实现形式，之后会系统性总结原理。假如现在有一个词典如下，输入的待编码单词是'moutain</w>'，
vocab = {'n': 4150, 's</w>': 4698, 'in': 3363, 'ta': 1009, 'ou': 3936, 'm': 7476}

In [150]:
# 第1步：先按照字符数量对词典中的token长度进行倒排。
def measure_token_length(token):
    if token[-4:] == '</w>':
        return len(token[:-4]) + 1
    else:
        return len(token)

def get_tokens_from_vocab(vocab):
    tokens_frequencies = collections.defaultdict(int)
    for word, freq in vocab.items():
        word_tokens = word.split()
        for token in word_tokens:
            tokens_frequencies[token] += freq
    sorted_tokens_tuple = sorted(tokens_frequencies.items(), key=lambda item: (measure_token_length(item[0]), item[1]),
                                 reverse=True)
    return [token for (token, freq) in sorted_tokens_tuple]

vocab = {'n': 4150, 's</w>': 4698, 'in': 3363, 'ta': 1009, 'ou': 3936, 'm': 7476}
sorted_tokens = get_tokens_from_vocab(vocab)
print("sorted_tokens:",sorted_tokens)

sorted_tokens: ['s</w>', 'ou', 'in', 'ta', 'm', 'n']


In [164]:
"""
第2步：从左到右拆分待编码单词，使得拆分后的子串总数尽量少，且子串都在词典中出现过。
详细流程为：待编码单词从左到右迭代，在词典中依次找到目标token，满足条件为尽量长且能作为待编码单词的子串。如果在词典中找不到作为待编码单词子串的token，就用<unk>候补。这样找到的子串组合即为待编码单词的编码结果。
如果采用暴力遍历的方式，算法时间复杂度非常高，BPE的做法是采用中序遍历解决问题，具体实现方式如下所示。
"""
def tokenize_word(string, sorted_tokens, unknown_token='</u>'):
    if string == '':
        return []
    if sorted_tokens == []:
        return [unknown_token]
    print("sorted_tokens:",sorted_tokens)
    string_tokens = []
    for i in range(len(sorted_tokens)):
        token = sorted_tokens[i]
        token_reg = re.escape(token.replace('.', '[.]'))
        matched_positions = [(m.start(0), m.end(0)) for m in re.finditer(token_reg, string)]
        print("token_reg:", token_reg, " matched_positions:", matched_positions)
        if len(matched_positions) == 0:
            continue
        substring_end_positions = [matched_position[0] for matched_position in matched_positions]

        print("substring_end_positions:", substring_end_positions) # 其实list的长度=1
        # 整体逻辑为中序遍历
        substring_start_position = 0
        for substring_end_position in substring_end_positions:
            ## 先遍历左子树
            substring = string[substring_start_position:substring_end_position]
            string_tokens += tokenize_word(string=substring, sorted_tokens=sorted_tokens[i + 1:],
                                           unknown_token=unknown_token)

            ## 后遍历根节点
            string_tokens += [token]
            substring_start_position = substring_end_position + len(token)

        ## 最后遍历右子树
        remaining_substring = string[substring_start_position:]
        string_tokens += tokenize_word(string=remaining_substring, sorted_tokens=sorted_tokens[i + 1:],
                                       unknown_token=unknown_token)
        break
    return string_tokens

In [165]:
# 这里总结编码的整体流程：
#对词典中token按照字符长度进行倒排
#从左到右拆分待编码单词，使得拆分后的子串总数尽量少，且子串都在词典中出现过。拆分后的子串组合即为待编码单词的编码结果。

word_given = 'inmountains</w>'
encodes = tokenize_word(string=word_given, sorted_tokens=sorted_tokens, unknown_token='</u>')

sorted_tokens: ['s</w>', 'ou', 'in', 'ta', 'm', 'n']
token_reg: s</w>  matched_positions: [(10, 15)]
substring_end_positions: [10]
sorted_tokens: ['ou', 'in', 'ta', 'm', 'n']
token_reg: ou  matched_positions: [(3, 5)]
substring_end_positions: [3]
sorted_tokens: ['in', 'ta', 'm', 'n']
token_reg: in  matched_positions: [(0, 2)]
substring_end_positions: [0]
sorted_tokens: ['ta', 'm', 'n']
token_reg: ta  matched_positions: []
token_reg: m  matched_positions: [(0, 1)]
substring_end_positions: [0]
sorted_tokens: ['in', 'ta', 'm', 'n']
token_reg: in  matched_positions: [(3, 5)]
substring_end_positions: [3]
sorted_tokens: ['ta', 'm', 'n']
token_reg: ta  matched_positions: [(1, 3)]
substring_end_positions: [1]
sorted_tokens: ['m', 'n']
token_reg: m  matched_positions: []
token_reg: n  matched_positions: [(0, 1)]
substring_end_positions: [0]


In [160]:
print("encodes:",encodes)

encodes: ['in', 'm', 'ou', 'n', 'ta', 'in', 's</w>']
